# Using `langchain` (Part 2)

### Story creation chain examples from Ch.4 of _Prompt Engineering_ book

* https://learning.oreilly.com/library/view/prompt-engineering-for/9781098153427/ch04.html#id260

![](img/story_creation_chain.png)

### Setup

* Run the following cells to set up the notebook environment

In [1]:
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.runnables import (
                                        RunnablePassthrough,
                                        RunnableLambda,
                                        RunnableParallel
)

from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser


from operator import itemgetter

from dotenv import load_dotenv
import os

In [2]:
# Load API key
_ = load_dotenv()

### 1. Story creation prompt template

In [3]:
character_generation_prompt = ChatPromptTemplate.from_template(
    """I want you to brainstorm 4 characters for my short story. The
    genre is {genre}. Each character must have a Name and a Biography.
    You must provide a name and biography for each character, this is very
    important!
    ---
    Example response:
    Name -- CharWiz, Biography -- A wizard who is a master of magic.
    Name -- CharWar, Biography -- A warrior who is a master of the sword.
    ---
    Characters: """
)

plot_generation_prompt = ChatPromptTemplate.from_template(
    """Given the following characters and the genre, create an effective
    plot for a short story:
    Characters:
    {characters}
    ---
    Genre: {genre}
    ---
    Plot: """
    )

scene_generation_plot_prompt = ChatPromptTemplate.from_template(
    """Act as an effective content creator.
    Given multiple characters and a plot you are responsible generating
    the various scenes for each act.

    You must de-compose the plot into multiple effective scenes:
    ---
    Characters:
    {characters}
    ---
    Genre: {genre}
    ---
    Plot: {plot}
    ---
    Example response:
    Scenes:
    Scene 1: Some text here.
    Scene 2: Some text here.
    Scene 3: Some text here.
    ----
    Scenes:
    """
)

In [4]:
print(character_generation_prompt.invoke({'genre': 'horror'}).messages[0].content)

I want you to brainstorm 4 characters for my short story. The
    genre is horror. Each character must have a Name and a Biography.
    You must provide a name and biography for each character, this is very
    important!
    ---
    Example response:
    Name -- CharWiz, Biography -- A wizard who is a master of magic.
    Name -- CharWar, Biography -- A warrior who is a master of the sword.
    ---
    Characters: 


#### Using passthrough and `itemgetter`

In [5]:
chain = RunnablePassthrough() | {
    "genre": itemgetter("genre"),
  }

chain.invoke({"genre": "fantasy"})
# {'genre': 'fantasy'}

{'genre': 'fantasy'}

In [6]:
master_chain = RunnablePassthrough() | {
    "genre": itemgetter("genre"),
    "upper_case_genre": lambda x: x["genre"].upper(),
    "lower_case_genre": RunnableLambda(lambda x: x["genre"].lower()),
}

### Subchains for characters, plot and scenes

* Define the sub chains

In [7]:
# Create the chat model:
model = ChatOpenAI()

# Create the sub-chains:
character_generation_chain = ( character_generation_prompt
| model
| StrOutputParser() )

plot_generation_chain = ( plot_generation_prompt
| model
| StrOutputParser() )

scene_generation_plot_chain = ( scene_generation_plot_prompt
| model
| StrOutputParser()  )

#### Call the characters generation chain

In [8]:
characters = character_generation_chain.invoke({'genre': 'horror'})

In [9]:
print(characters)

1. Name -- Dr. Abigail Blackwell, Biography -- A brilliant and ambitious scientist who will stop at nothing to achieve her goals, even if it means crossing ethical boundaries. She is known for her cold demeanor and ruthless determination.

2. Name -- Marcus Bennett, Biography -- A former soldier plagued by PTSD and haunted by his experiences in combat. He is struggling to adjust to civilian life and is easily triggered by loud noises and crowded spaces. Despite his tough exterior, he is deeply traumatized and fragile.

3. Name -- Emily Woods, Biography -- A young and naive college student who is eager to explore the paranormal. She is fascinated by ghosts and the supernatural, often putting herself in dangerous situations in pursuit of a thrilling experience. Her curiosity and recklessness make her a liability to herself and others.

4. Name -- Reverend Johnathan Hayes, Biography -- A charismatic preacher with a dark secret. He is a master manipulator who uses his charm and influence t

#### Call the plot generation chain

In [ ]:
plot = plot_generation_chain.invoke({'characters': characters, 'genre': 'horror'})

In [ ]:
print(plot)

#### Call the scenes generation chain

In [ ]:
scenes = scene_generation_plot_chain.invoke({
    'genre': 'horror',
    'characters': characters,
    'plot': plot
})

In [ ]:
print(scenes)

### Build the master chain

In [ ]:
master_chain = (
    {"characters": character_generation_chain, "genre":
    RunnablePassthrough()}
    | RunnableParallel(
        characters=itemgetter("characters"),
        genre=itemgetter("genre"),
        plot=plot_generation_chain,
    )
    | RunnableParallel(
        characters=itemgetter("characters"),
        genre=itemgetter("genre"),
        plot=itemgetter("plot"),
        scenes=scene_generation_plot_chain,
    )
)

In [ ]:
story_result = master_chain.invoke({"genre": "Under 5s children horror mystery"})

In [ ]:
story_result.keys()

In [ ]:
print(story_result['genre'])

In [ ]:
print(story_result['characters'])

In [ ]:
print(story_result['plot'])

In [ ]:
print(story_result['scenes'])